# GLM，ロジスティック回帰モデル
statesmodelsの参照：<br>
- Generalized Linear Models https://www.statsmodels.org/stable/glm.html  
- GLM: Binomial response data https://www.statsmodels.org/stable/examples/notebooks/generated/glm.html  
- Generalized Linear Models (Formula) https://www.statsmodels.org/stable/examples/notebooks/generated/glm_formula.html  
- Influence Measures for GLM Logit https://www.statsmodels.org/dev/examples/notebooks/generated/influence_glm_logit.html

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import scipy.stats as stats
import pandas as pd

import statsmodels.api as sm
import statsmodels.formula.api as smf

import matplotlib.pyplot as plt
%matplotlib inline

#### 二項分布に従う確率変数とそのヒストグラム
scipy.stats.binom https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.binom.html<br>
上のサイト内の二項分布の式$f(k)$は，1/0が出力であり1が出る確率をpとおいたとき，n回の試行のうち1がk回出る確率を表す。<br>
下記のstats.binom.rvs(n, p, size=N)は，n回試行して1が出る回数k（$0 \le k \le n$）を求める。大文字のNは，この試行回数である。<br>
例えば，p=1/2ならば，1が出る確率は50\%だから，ヒストグラムのピークは真ん中（n/2）になる。<br>
また，$E[X] = np, \; V[X]=np(1-p)$

In [ ]:
np.random.seed(123)
N = 100
n, p = 4, 0.5 # p:生存確率
rv = stats.binom.rvs(n, p, size=N) # 
print(rv)
print('rv mean=',rv.mean(), '  rv var      =',rv.var())
print('E[x]=np=',n*p, '   V[x]=np(1-p)=',n*p*(1-p))
bins=[0, 1, 2, 3, 4, 5, 6]
fig = plt.subplots(figsize=(5,2))
plt.hist(rv, bins=bins, align='left', range=(-1,6), rwidth=0.5, color='gray')
plt.xlabel('n')
plt.ylabel('frequency')
plt.grid()

## GLMのための人工データ作成
人工データ作成のため，初めに，ロジスティック曲線を眼で確認しながら，試行錯誤的に定める。<br>
zは線形予測子である。

In [ ]:
xend, N = 10.0, 100
b0, b1 = -15.0, 3.0
#b0, b1 = -50.0, 10.0
#b0, b1 = -30.0, 8.0
x = np.linspace(0.0, xend, N)
z = b0 + b1*x
p = 1/(1+np.exp(-z))

fig = plt.subplots(figsize=(5,2))
plt.plot(x, p)
plt.xlabel('x')
plt.grid()

上記で，xは系列として作成された。<br>
１番目に，系列である必要は無いことを明示するため，このxのレンジ[0,xend]内で，xの値をランダムに発生させる。これをxrvsに格納する。<br>
２番目に，pを多少乱す。この理由は，現実に実験したとき，上記のような綺麗なデータは取れないであろうということと，statsmodelsのGLMの計算の安定性の確保のためである。<br>
これを行うのがnp.random.rand(N)*coef，であり，coefの値は試行錯誤で定めた。<br>
３番目に，問題に適するよう，適当なしきい値の下で，0/1のデータに変換して格納したのがyrvsである。

In [ ]:
np.random.seed(123)

xrvs = np.random.rand(N)*xend # range [0,xend]
z = b0 + b1*xrvs
p = 1/(1+np.exp(-z)) # probability
p = p + np.random.rand(N)*0.52 # ２番目のpを乱す。左の係数は試行錯誤で決めた
yrvs = np.where(p < 0.5, 0.0, 1.0) # 2値化
#yrvs = p

fig = plt.subplots(figsize=(6,3))
plt.scatter(xrvs,p, c='gray')
plt.scatter(xrvs,yrvs, c='b')
plt.xlabel('x')
plt.grid()
plt.show()

参考までに，(xrvs, yrvs)を系列としてプロットすると次のようなグラフになる。

In [ ]:
fig = plt.subplots(figsize=(6,3))
plt.plot(xrvs,yrvs)
plt.show()

In [ ]:
df = pd.DataFrame({'y':yrvs, 'x':xrvs})
print(df.head())

fig, ax = plt.subplots(figsize=(6,3))
df.plot.scatter(x='x', y='y', ax=ax)
plt.show()

In [ ]:
glm_model = 'y  ~ x'
result_1 = smf.glm(formula=glm_model, data=df, family=sm.families.Binomial()).fit()
#result_1 = sm.GLM(df.y, df.x, family=sm.families.Binomial(link=sm.families.links.logit())).fit()
print(result_1.summary())

In [ ]:
b0, b1 = result_1.params[0], result_1.params[1]
y = 1/(1+np.exp(-b0 - b1*x))

fig = plt.subplots(figsize=(5,2))
plt.plot(x, y, c='k', label='p')
#plt.scatter(xrvs,p)
plt.scatter(xrvs,yrvs, c='w', ec='k', label='data', s=60)
plt.xlabel('x', fontsize=14)
plt.legend(loc='upper left', fontsize=14)
plt.grid()
#plt.savefig('fig_REG_GLM_Logistic03.pdf', bbox_inches='tight')
plt.show()

#### 備考
pを確率と考えるならば，それはロジスティック分布の累積分布関数であり，その確率密度関数は次のサイトを参照。<br>
https://ja.wikipedia.org/wiki/ロジスティック分布<br>
この分布に従う確率変数を発生するライブラリが無い場合には，累積分布関数（ここではpのこと）を用いて，逆関数法を用いればよい。<br>
https://ja.wikipedia.org/wiki/逆関数法